# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [57]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
import create_tables
create_tables.main()

In [ ]:
host = '127.0.0.1'
dbname = 'sparkifydb'
user = 'student' 
password = 'student'
conn = psycopg2.connect(f"host={host} dbname={dbname} user={user} password={password}")
cur = conn.cursor()

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
song_files = get_files('data/song_data')

In [6]:
filepath = song_files[0]

In [ ]:
# df = pd.read_json(filepath, lines=True)
# print(df.shape)
# df.head()

In [10]:
df_list = []
for filepath in song_files:
    df = pd.read_json(filepath, lines=True)
    df_list.append(df)
df = pd.concat(df_list)
print(df.shape)
df.head()

(72, 10)


,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,AR8IEZO1187B99055E,NaN,,NaN,Marc Shaiman,149.86404,1,SOINLJW12A8C13314C,City Slickers,2008
0,AR558FS1187FB45658,NaN,,NaN,40 Grit,75.67628,1,SOGDBUF12A8C140FAA,Intro,2003
0,ARVBRGZ1187FB4675A,NaN,,NaN,Gwen Stefani,290.55955,1,SORRZGD12A6310DBC3,Harajuku Girls,2004
0,ARNF6401187FB57032,40.79086,"New York, NY [Manhattan]",-73.96644,Sophie B. Hawkins,305.16200,1,SONWXQJ12A8C134D94,The Ballad Of Sleeping Beauty,1994
0,AREVWGE1187B9B890A,-13.44200,Noci (BA),-41.99520,Bitter End,282.43546,1,SOFCHDR12AB01866EF,Living Hell,0


In [41]:
df.isnull().sum()

artist_id            0
artist_latitude     41
artist_location      0
artist_longitude    41
artist_name          0
duration             0
num_songs            0
song_id              0
title                0
year                44
loc                 28
location             0
dtype: int64

In [47]:
import numpy as np
df['artist_location'].replace('', np.nan, inplace=True)
df['year'].replace(0, np.nan, inplace=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year,loc,location
0,AR8IEZO1187B99055E,NaN,NaN,NaN,Marc Shaiman,149.86404,1,SOINLJW12A8C13314C,City Slickers,2008.0,NaN,False
0,AR558FS1187FB45658,NaN,NaN,NaN,40 Grit,75.67628,1,SOGDBUF12A8C140FAA,Intro,2003.0,NaN,False
0,ARVBRGZ1187FB4675A,NaN,NaN,NaN,Gwen Stefani,290.55955,1,SORRZGD12A6310DBC3,Harajuku Girls,2004.0,NaN,False
0,ARNF6401187FB57032,40.79086,"New York, NY [Manhattan]",-73.96644,Sophie B. Hawkins,305.16200,1,SONWXQJ12A8C134D94,The Ballad Of Sleeping Beauty,1994.0,"New York, NY [Manhattan]",False
0,AREVWGE1187B9B890A,-13.44200,Noci (BA),-41.99520,Bitter End,282.43546,1,SOFCHDR12AB01866EF,Living Hell,NaN,Noci (BA),False


In [ ]:
df.isnull().sum()

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [9]:
song_data = df[['song_id','title','artist_id','year','duration']].values[0]
song_data

array(['SOINLJW12A8C13314C', 'City Slickers', 'AR8IEZO1187B99055E', 2008,
       149.86404], dtype=object)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [10]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [11]:
artist_data = df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].values[0]
artist_data

array(['AR8IEZO1187B99055E', 'Marc Shaiman', '', nan, nan], dtype=object)

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [12]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [48]:
log_files = get_files('data/log_data')

In [12]:
# filepath = log_files[0]

In [13]:
# df = pd.read_json(filepath, lines=True)
# print(df.shape)
# df.head(1)

(366, 18)


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


In [50]:
df_list = []
for filepath in log_files:
    df = pd.read_json(filepath, lines=True)
    df_list.append(df)
df = pd.concat(df_list)
print(df.shape)
df.head()

(8056, 18)


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,1543449841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,1543449895796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,1543450033796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


In [52]:
df = df[df['page']=='NextSong']
df.isnull().sum()

artist           0
auth             0
firstName        0
gender           0
itemInSession    0
lastName         0
length           0
level            0
location         0
method           0
page             0
registration     0
sessionId        0
song             0
status           0
ts               0
userAgent        0
userId           0
dtype: int64

In [58]:
df['location'].value_counts()

San Francisco-Oakland-Hayward, CA                 691
Portland-South Portland, ME                       665
Lansing-East Lansing, MI                          557
Chicago-Naperville-Elgin, IL-IN-WI                475
Atlanta-Sandy Springs-Roswell, GA                 456
Waterloo-Cedar Falls, IA                          397
Lake Havasu City-Kingman, AZ                      321
Tampa-St. Petersburg-Clearwater, FL               307
San Jose-Sunnyvale-Santa Clara, CA                292
Sacramento--Roseville--Arden-Arcade, CA           270
New York-Newark-Jersey City, NY-NJ-PA             262
Janesville-Beloit, WI                             248
Birmingham-Hoover, AL                             223
Winston-Salem, NC                                 213
Red Bluff, CA                                     201
Marinette, WI-MI                                  169
Augusta-Richmond County, GA-SC                    140
Detroit-Warren-Dearborn, MI                        76
Houston-The Woodlands-Sugar 

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [16]:
df = df[df['page']=='NextSong']
print(df.shape)
df.head(1)

(319, 18)


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


In [17]:
df['datetime'] = pd.to_datetime(df['ts'], unit='ms')
df['hour'] = df['datetime'].dt.hour
df['day'] = df['datetime'].dt.day
df['week'] = df['datetime'].dt.week
df['month'] = df['datetime'].dt.month
df['year'] = df['datetime'].dt.year
df['weekday'] = df['datetime'].dt.weekday
df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,datetime,hour,day,week,month,year,weekday
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73,2018-11-29 00:00:57.796,0,29,48,11,2018,3


In [ ]:
# convert timestamp column to datetime
t = pd.to_datetime(df['ts'], unit='ms')

In [32]:
# insert time data records
# set time_data to a list containing these values in order
time_data = list(zip(list(df['ts']), list(t.dt.hour), list(t.dt.day), list(t.dt.week), list(t.dt.month), list(t.dt.year), list(t.dt.weekday)))
# Specify labels for these columns and set to column_labels
column_labels = ['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']

In [33]:
time_df = pd.DataFrame(time_data, columns=column_labels, dtype=int)
print(time_df.shape)
time_df.head()

(319, 7)


,start_time,hour,day,week,month,year,weekday
0,1543449657796,0,29,48,11,2018,3
1,1543449690796,0,29,48,11,2018,3
2,1543449841796,0,29,48,11,2018,3
3,1543449895796,0,29,48,11,2018,3
4,1543450033796,0,29,48,11,2018,3


In [19]:
time_df = df[['ts', 'hour', 'day', 'week', 'month', 'year', 'weekday']]
time_df = time_df.rename(columns={'ts':'start_time'})
print(time_df.shape)
time_df.head(1)

(319, 7)


,start_time,hour,day,week,month,year,weekday
0,1543449657796,0,29,48,11,2018,3


In [20]:
time_df.dtypes

start_time    int64
hour          int64
day           int64
week          int64
month         int64
year          int64
weekday       int64
dtype: object

In [21]:
time_df.duplicated('start_time').value_counts()

False    319
dtype: int64

In [22]:
# Drop all duplicates on column 'start_time'
time_df = time_df.drop_duplicates(subset=['start_time'], keep=False)
time_df.shape

(319, 7)

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [23]:
# !pip install psycopg2-binary

In [24]:
from sqlalchemy import create_engine

In [25]:
# Append entire dataframe to postgres database table
# url = f"postgres://{user}:{password}@{host}:5432/{dbname}"
# engine = create_engine(url)
# time_df.to_sql(name="time", con=engine, if_exists="append", index=False)

In [34]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [26]:
df["userId"] = pd.to_numeric(df["userId"])
user_df = df[['userId','firstName','lastName','gender','level']]
user_df = user_df.rename(columns={'userId':'user_id','firstName':'first_name','lastName':'last_name'})
print(user_df.shape)
user_df.head(2)

(319, 5)


,user_id,first_name,last_name,gender,level
0,73,Jacob,Klein,M,paid
1,24,Layla,Griffin,F,paid


In [27]:
user_df.dtypes

user_id        int64
first_name    object
last_name     object
gender        object
level         object
dtype: object

In [28]:
len(user_df['user_id'].value_counts())

27

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [29]:
# Drop duplicates except for the last occurrence on column 'userId'
user_df_dd = user_df.drop_duplicates(subset=['user_id'], keep='last')
print(user_df_dd.shape)
user_df_dd

(27, 5)


,user_id,first_name,last_name,gender,level
7,73,Jacob,Klein,M,paid
21,50,Ava,Robinson,F,free
34,24,Layla,Griffin,F,paid
39,54,Kaleb,Cook,M,free
47,14,Theodore,Harris,M,free
49,98,Jordyn,Powell,F,free
53,78,Chloe,Roth,F,free
55,101,Jayden,Fox,M,free
56,10,Sylvie,Cruz,F,free
57,52,Theodore,Smith,M,free


In [30]:
# Append entire dataframe to postgres database table
url = f"postgres://{user}:{password}@{host}:5432/{dbname}"
engine = create_engine(url)
user_df_dd.to_sql(name="users", con=engine, if_exists="append", index=False)

In [31]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [32]:
df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,datetime,hour,day,week,month,year,weekday
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73,2018-11-29 00:00:57.796,0,29,48,11,2018,3


In [15]:
for index, row in df.iterrows():
    if index == 3: break
    print(f"song:{row.song}, artist:{row.artist}, length:{row.length}")

song:Ain't No Sunshine, artist:Sydney Youngblood, length:238.07955
song:My Advice 2 You (Explicit), artist:Gang Starr, length:151.92771
song:My First Kiss (Feat. Ke$ha) [Album Version], artist:3OH!3, length:192.522
song:The Girl and The Robot, artist:RÃÂ¶yksopp, length:369.81506
song:Too Shy, artist:Kajagoogoo, length:223.55546
song:Potential, artist:The Jimmy Castor Bunch, length:306.54649
song:The Great Satan (What Would Satan Do Mix), artist:Ministry, length:186.69669
song:The Screamer, artist:Bouncing Souls, length:117.31546
song:Knife Going In (Album Version), artist:Tegan And Sara, length:129.67138
song:The Greatest View (Album Version), artist:Silverchair, length:242.96444


In [33]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    # timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent
    songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [34]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.